<a href="https://colab.research.google.com/github/trotsak/text/blob/main/text_ml_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Название проекта: Классификация текстов для интернет-магазина «Викишоп»**.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [50]:
!pip install -q optuna catboost lightgbm xgboost

In [51]:
# работа с операционной системой
import os

## Импорт библиотек для анализа и визуализации данных

# работа с данными в формате таблиц
import pandas as pd

# работа с многомерными массивами
import numpy as np

# Импортируем функцию sqrt (квадратный корень) из модуля math
from math import sqrt

# Импортируем функцию autocorrelation_plot из библиотеки pandas.plotting
from pandas.plotting import autocorrelation_plot

# визуализация данных
import matplotlib.pyplot as plt

# импорт функции display для отображения датафреймов и других объектов в Jupyter Notebook
from IPython.display import display

# Импорт модуля re для работы с регулярными выражениями
import re

# импорт конфигурационных параметров для настройки отображения графиков
from matplotlib import rcParams, rcParamsDefault

# расширенные возможности визуализации
import seaborn as sns

# Импортируем модуль time для работы со временем
import time

## Импорт библиотек для статистического анализа

# статистические функции
from scipy import stats as st

# Импортируем функцию adfuller из модуля statsmodels.tsa.stattools
from statsmodels.tsa.stattools import adfuller

# Импорт функции для выполнения теста KPSS (Kwiatkowski-Phillips-Schmidt-Shin)
# для проверки стационарности временного ряда
from statsmodels.tsa.stattools import kpss

# Импортируем функции для визуализации автокорреляции и частичной автокорреляции
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# импорт библиотеки для статистического анализа
import scipy.stats as stats

# Импорт функции seasonal_decompose из библиотеки statsmodels.
from statsmodels.tsa.seasonal import seasonal_decompose

## Импорт библиотек для машинного обучения

# Импортируем ColumnTransformer для предобработки данных по столбцам
from sklearn.compose import ColumnTransformer

# Импортируем Pipeline для создания конвейера обработки данных и обучения модели
from sklearn.pipeline import Pipeline

# Импортируем функцию set_config из библиотеки scikit-learn
from sklearn import set_config


# общая библиотека машинного обучения
import sklearn

# разделение данных и оценка моделей
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RandomizedSearchCV

# кодирование категориальных переменных и стандартизация
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

# Импортируем класс LinearRegression из библиотеки sklearn (подмодуля linear_model)
# from sklearn.linear_model import LinearRegression

# Импортируем класс RandomForestRegressor из библиотеки sklearn (подмодуля ensemble)
# from sklearn.ensemble import RandomForestRegressor

# Импортируем KNNImputer из библиотеки sklearn.impute
# from sklearn.impute import KNNImputer

# Импорт модели LightGBM
# from lightgbm import LGBMRegressor

# Импортируем lightgbm как lgb
import lightgbm as lgb

# Импорт функции mean_squared_error из библиотеки scikit-learn
from sklearn.metrics import mean_squared_error, make_scorer

## Импорт библиотеки для обработки предупреждений

# управление предупреждениями
import warnings

# игнорировать предупреждения (если нужно)
# warnings.filterwarnings('ignore')

import optuna
import re
import string
import requests
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
wnl = WordNetLemmatizer()


stopwords = set(nltk_stopwords.words('english'))

from sklearn.metrics import f1_score


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import spacy

nlp = spacy.load("en_core_web_sm")

from tqdm import tqdm
tqdm.pandas()

# Создание общего прогресс-бара для apply
tqdm.pandas(desc="Общий прогресс")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
class Color:

    """
    Класс для хранения цветовых кодов для форматирования текстов в терминале.
    """

    PURPLE = '\033[95m'      # Фиолетовый цвет
    CYAN = '\033[96m'        # Бирюзовый цвет
    DARK_CYAN = '\033[36m'   # Темно-бирюзовый цвет
    BLUE = '\033[94m'        # Синий цвет
    GREEN = '\033[92m'       # Зеленый цвет
    YELLOW = '\033[93m'      # Желтый цвет
    RED = '\033[91m'         # Красный цвет
    BOLD = '\033[1m'         # Жирный текст
    UNDERLINE = '\033[4m'    # Подчеркнутый текст
    END = '\033[0m'          # Сброс формата текста


In [53]:
# определение констант
RANDOM_STATE = 42
TEST_SIZE = 0.10
CV_COUNTS=5

In [54]:
# системные настройки
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Настроим параметры отображения графиков в Matplotlib для лучшей визуализации и качества изображения.

In [55]:
# установка стиля графиков на основе библиотеки Seaborn
sns.set_style('whitegrid')

# Включаем отображение объектов scikit-learn в виде диаграммы
set_config(display='diagram')

# установка формата изображения SVG для обеспечения более четкого и качественного изображение графиков.
%config InlineBackend.figure_format = 'svg'

# масштабный фактор, который будет использоваться для изменения параметра dpi.
factor = 0.8

# извлечение значения по умолчанию для точек на дюйм (dpi) из настроек Matplotlib.
default_dpi = rcParamsDefault['figure.dpi']

# установка разрешения (dpi) для всех фигур путём умножения значения dpi на масштабный фактор.
rcParams['figure.dpi'] = default_dpi*factor

# включение отображения графиков в Jupyter
%matplotlib inline

# установка размера диаграмм
rcParams['figure.figsize'] = [12.0, 6.0]

## Подготовка

### Загрузим данные из csv-файла в датафрейм.

In [56]:
## считывание данных из csv-файлов в датафреймы

# назначение путей к файлам
file_paths = {
    'toxic_comments': '/datasets/ toxic_comments.csv'
}

# словарь для хранения загруженных данных
dataframes = {}

# проход по всем файлам
for name, path in file_paths.items():
    try:
        if os.path.exists(path):
            dataframes[name] = pd.read_csv(path)
            print(f'Файл {path} загружен из локального пути.')
        else:
            url = f'https://code.s3.yandex.net/datasets/{name}.csv'
            dataframes[name] = pd.read_csv(url)
            print(f'Файл {path} загружен из URL.')
    except Exception as e:
        print(f'Не удалось загрузить {path}: {e}')

# присваивание загруженным датафреймам отдельных переменных
comments_data = dataframes['toxic_comments']

Файл /datasets/ toxic_comments.csv загружен из URL.


## Анализ

### Изучим общую информацию о полученном датафрейме
Создадим функцию `data_info` для вывода общей информации по датафрейму.

In [57]:
# создание функции для вывода общей информации по датафрейму
def data_info(data, dataframe_name):
    """
    Отображает общую информацию о переданном датафрейме.

    Функция выполняет следующие операции:
    1. Отображение первых нескольких строк датафрейма.
    2. Вывод общей информации о датафрейме, включая типы данных и количество ненулевых значений.
    3. Отображение статистического описания числовых столбцов.
    4. Подсчет и вывод количества пропущенных значений в каждом столбце.
    5. Вывод количества явных дубликатов в датафрейме.
    6. Отображение списка названий столбцов в датафрейме.
    7. Вывод уникальных значений для столбцов с типом данных 'object'.
    8. Вывод числа уникальных значений для каждого столбца.
    9. Вывод числа дублей для каждого столбца.

    Параметры:
    ----------
    data : pandas.DataFrame
        Датафрейм, для которого необходимо вывести информацию.
    dataframe_name : str
        Имя датафрейма (для отображения в выводе).
    """

    # отображение первых несколько строк датафрейма
    print(Color.BOLD + f"Первые строки датафрейма {dataframe_name}:\n" + Color.END)
    display(data.head())
    print()

    # вывод информацию о датафрейме, включая типы данных и количество ненулевых значений
    print(Color.BOLD + f"Общая информация о датафрейме {dataframe_name}:\n" + Color.END)
    data.info()
    print()

    # отображение статистического описания числовых столбцов датафрейма
    print(Color.BOLD + f"Статистическое описание числовых столбцов датафрейма {dataframe_name}:\n" + Color.END)
    display(data.describe())
    print()

    # отображение количества пропущенных значений в каждом столбце
    print(Color.BOLD + f"Количества пропущенных значений в каждом столбце датафрейма {dataframe_name}:\n" + Color.END)
    display(data.isna().sum())
    print()

    # вывод количества явных дубликатов в датафрейме
    print(f'Количество явных дубликатов в датафрейме: {Color.RED}{data.duplicated().sum()}{Color.END}.')
    print()

    # отображение списка названий столбцов в датафрейме
    print(Color.BOLD + f"Cписок названий столбцов в датафрейме {dataframe_name}:\n" + Color.END)
    display(data.columns.tolist())

    # отображение всех уникальных значений и их количества в столбцах типа 'object'
    for i in data.columns:
        if data[i].dtype == 'object':
            unique_values = data[i].unique()
            num_unique_values = len(unique_values)
            num_duplicates = data[i].duplicated().sum()
            print(f'В столбце {Color.BOLD}\'{i}\'{Color.END} содержится {num_unique_values} уникальных значений: \
            {Color.BOLD}{unique_values}{Color.END}')
            print(f'Число дублей в столбце {Color.BOLD}\'{i}\'{Color.END}: {Color.RED}{num_duplicates}{Color.END}')

#### Общая информация о датафреме `taxi_data`:

In [58]:
# получение общей информации по датафрейму с помощью функции data_info
data_info(comments_data, 'comments_data')

Первые строки датафрейма comments_data:



,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0



Общая информация о датафрейме comments_data:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB

Статистическое описание числовых столбцов датафрейма comments_data:



,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000



Количества пропущенных значений в каждом столбце датафрейма comments_data:



,0
Unnamed: 0,0
text,0
toxic,0



Количество явных дубликатов в датафрейме: 0.

Cписок названий столбцов в датафрейме comments_data:



['Unnamed: 0', 'text', 'toxic']

В столбце 'text' содержится 159292 уникальных значений:             ["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  W

**Выводы:**

В датафрейме `taxi_data` содержится 4416 строк и 2 колонки с численными и временными типами данных. Индекс отсортирован в монотонном порядке.

Согласно документации колонки содержат следующую информацию:

- `datetime` - время заказов такси,
- `num_orders` - количество заказов такси.

Подписи колонок соответствуют хорошему стилю написания, пропущенные значения и аномалии отсутствуют, даты расположены в хронологическом порядке, присутствуют дубли, но в данной ситуации это нормально, удалять не будем, количество заказов такси может повторяться.

За 1-часовой период: в среднем 84 заказа, типичная нагрузка в 75% случаев составляет до 107 заказов, изменчивость умеренная - стандартное отклонение 45,  максимальное число заказов значительно отличается от значения среднего и медианы - 462, что указывает на наличие всплесков спроса; минимальное число заказов - 0.

In [59]:
comments_data = comments_data.drop('Unnamed: 0', axis=1)
display(comments_data.head())
comments_data.info()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [60]:
# Оптимизируем тип данных признака 'toxic' (int64 переведем в формат uint8):
comments_data['toxic'] = comments_data['toxic'].astype('uint8')

In [61]:
comments_data.groupby('toxic')['text'].count()

,text
toxic,
0,143106
1,16186


In [62]:
# from tqdm import tqdm
# def lemmatize_text(text):

#     # Удаляем символы, не относящиеся к русскому алфавиту
#     sub_text = re.sub(r'[^a-zA-Z ]', ' ', text)
#     join_text = " ".join(sub_text.split())

#     doc = nlp(join_text)

#     lemmatized_tokens = []


#     # Добавляем прогресс-бар с помощью tqdm
#     for token in doc:
#         lemmatized_tokens.append(token.lemma_)

#     # Удаляем лишние пробелы и объединяем лемматизированные токены
#     lem_text = " ".join(lemmatized_tokens)

#     return lem_text

In [63]:
# lemmatize_text("The striped bats are hanging on their feet for best")

In [64]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [65]:
disabled_pipes = [ "parser",  "ner"]
nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)

In [ ]:
lemm_texts = []

for doc in tqdm(nlp.pipe(comments_data['text'].values, disable = ['ner', 'parser']),
                total=comments_data.shape[0],
                desc="Обработка текста"
                ):
        lemm_text = " ".join([i.lemma_ for i in doc])
        lemm_texts.append(lemm_text)

Обработка текста:   0%|          | 362/159292 [00:04<21:04, 125.69it/s]

In [ ]:
# import spacy

# # Загрузка модели Spacy для английского языка
# nlp = spacy.load("en_core_web_sm")

# Исходное предложение
sentence = "The striped bats are hanging on their feet for best"

# Применение модели Spacy
doc = nlp(sentence)

# Лемматизированный текст
lemm_text = " ".join([token.lemma_ for token in doc])

print("Исходный текст:", sentence)
print("Лемматизированный текст:", lemm_text)

In [ ]:
# Проверяем первые N строк
for i, lemm_text in enumerate(lemm_texts[:5]):
    print(f"Текст {i + 1}: {comments_data['text'].iloc[i]}")
    print(f"Лемматизированный: {lemm_text}")
    print("-" * 50)

In [ ]:
# Результат
comments_data['lemm_texts'] = lemm_texts
# comments_data.head()
lemm_texts.head()

In [ ]:
comments_data.head()

In [ ]:
# comments_data['lem_text'] = comments_data['text'].progress_apply(lemmatize_text)
# print(comments_data['lem_text'].head())